<a href="https://colab.research.google.com/github/lukiod/T2I-and-I2I-Report/blob/main/dreamdoodle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Create directories
!mkdir -p /content/DreamDoodleWebApp/static/uploads
!mkdir -p /content/DreamDoodleWebApp/static/results
!mkdir -p /content/DreamDoodleWebApp/templates

# Navigate into the main project directory
%cd /content/DreamDoodleWebApp

/content/DreamDoodleWebApp


In [2]:
# Install PyTorch matching Colab's CUDA version (usually handled automatically, but specific version helps)
# Check !nvidia-smi for CUDA version if needed, but 2.3.0 should work with recent Colab GPUs
!python3 -m pip install -q -U torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0

# Install diffusers and related HF libraries
!python3 -m pip install -q -U diffusers transformers accelerate Pillow matplotlib pandas numpy

# Install OneDiff/Nexfort stack
!python3 -m pip install -q -U nexfort torchao # <-- REMOVED ==0.1
!python3 -m pip install -q --pre onediff onediffx

# Install OneFlow (Optional, only if using --compiler oneflow)
# !python3 -m pip install -q oneflow
# Or specific wheel if needed:
# !python3 -m pip install -q -U --pre oneflow -f https://github.com/siliconflow/oneflow_releases/releases/expanded_assets/community_cu122

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [3]:
%%writefile generate.py

# ==================================================================
# PASTE THE ENTIRE CONTENT of your working generate.py script here
# ==================================================================
# Example start (replace with your full code):
# Default Settings (Mostly T2I oriented, override via command line)
# Default Settings (Mostly T2I oriented, override via command line)
DEFAULT_MODEL = "SG161222/RealVisXL_V4.0" # Default to a T2I XL model
DEFAULT_VARIANT = None
DEFAULT_CUSTOM_PIPELINE = None
DEFAULT_SCHEDULER = "EulerAncestralDiscreteScheduler"
DEFAULT_LORA = None
DEFAULT_CONTROLNET = None
DEFAULT_STEPS = 30
DEFAULT_PROMPT = "best quality, realistic, unreal engine, 4K, a cat sitting on human lap"
DEFAULT_NEGATIVE_PROMPT = ""
DEFAULT_SEED = 333
DEFAULT_WARMUPS = 1
DEFAULT_BATCH = 1
DEFAULT_HEIGHT = None # Auto-detect from model if None
DEFAULT_WIDTH = None  # Auto-detect from model if None
DEFAULT_INPUT_IMAGE = None # If provided, triggers I2I mode
DEFAULT_CONTROL_IMAGE = None
DEFAULT_OUTPUT_IMAGE = "generated_image.png" # Provide a default output name
DEFAULT_EXTRA_CALL_KWARGS = None # e.g., '{"strength": 0.75, "guidance_scale": 7.5}'
DEFAULT_CACHE_INTERVAL = 3
DEFAULT_CACHE_LAYER_ID = 0
DEFAULT_CACHE_BLOCK_ID = 0
DEFAULT_COMPILER = "nexfort"
DEFAULT_COMPILER_CONFIG = None
DEFAULT_QUANTIZE_CONFIG = None
DEFAULT_TASK = "auto" # Can be 'auto', 'text2image', 'image2image', 'instructpix2pix'

import os
import importlib
import inspect
import argparse
import time
import json
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
from diffusers.utils import load_image, is_accelerate_available, is_accelerate_version

# Required for onediffx/nexfort
from onediffx import compile_pipe, quantize_pipe

# --- Argument Parsing ---
def parse_args():
    parser = argparse.ArgumentParser(description="Generate images using diffusion models (T2I/I2I).")
    parser.add_argument("--model", type=str, default=DEFAULT_MODEL, help="Hugging Face model ID or local path.")
    parser.add_argument("--variant", type=str, default=DEFAULT_VARIANT, help="Model variant (e.g., 'fp16').")
    parser.add_argument("--custom-pipeline", type=str, default=DEFAULT_CUSTOM_PIPELINE, help="Custom pipeline class path.")
    parser.add_argument("--scheduler", type=str, default=DEFAULT_SCHEDULER, help="Scheduler name (e.g., 'EulerAncestralDiscreteScheduler', 'DPMSolverMultistepScheduler', 'none').")
    parser.add_argument("--lora", type=str, default=DEFAULT_LORA, help="Path or Hub ID for LoRA weights.")
    parser.add_argument("--controlnet", type=str, default=DEFAULT_CONTROLNET, help="Path or Hub ID for ControlNet model.")
    parser.add_argument("--steps", type=int, default=DEFAULT_STEPS, help="Number of inference steps.")
    parser.add_argument("--prompt", type=str, default=DEFAULT_PROMPT, help="Text prompt for generation/modification.")
    parser.add_argument("--negative-prompt", type=str, default=DEFAULT_NEGATIVE_PROMPT, help="Negative prompt.")
    parser.add_argument("--seed", type=int, default=DEFAULT_SEED, help="Random seed for generation. Set to None for random.")
    parser.add_argument("--warmups", type=int, default=DEFAULT_WARMUPS, help="Number of warmup runs before timing.")
    parser.add_argument("--batch", type=int, default=DEFAULT_BATCH, help="Number of images per prompt (batch size).")
    parser.add_argument("--height", type=int, default=DEFAULT_HEIGHT, help="Image height in pixels. Defaults to model's optimal size.")
    parser.add_argument("--width", type=int, default=DEFAULT_WIDTH, help="Image width in pixels. Defaults to model's optimal size.")
    parser.add_argument("--cache_interval", type=int, default=DEFAULT_CACHE_INTERVAL, help="DeepCache cache interval.")
    parser.add_argument("--cache_layer_id", type=int, default=DEFAULT_CACHE_LAYER_ID, help="DeepCache cache layer ID.")
    parser.add_argument("--cache_block_id", type=int, default=DEFAULT_CACHE_BLOCK_ID, help="DeepCache cache block ID.")
    parser.add_argument("--extra-call-kwargs", type=str, default=DEFAULT_EXTRA_CALL_KWARGS, help="JSON string of extra kwargs for the pipeline call (e.g., '{\"strength\": 0.8, \"guidance_scale\": 7.0}').")
    parser.add_argument("--input-image", type=str, default=DEFAULT_INPUT_IMAGE, help="Path or URL to the input image. If provided, enables Image-to-Image mode.")
    parser.add_argument("--control-image", type=str, default=DEFAULT_CONTROL_IMAGE, help="Path or URL to the ControlNet conditioning image.")
    parser.add_argument("--output-image", type=str, default=DEFAULT_OUTPUT_IMAGE, help="Path to save the generated image.")
    parser.add_argument("--throughput", action="store_true", help="Run throughput analysis.")
    parser.add_argument("--deepcache", action="store_true", help="Use DeepCache optimization (if supported by the model/pipeline).")
    parser.add_argument(
        "--task",
        type=str,
        default=DEFAULT_TASK,
        choices=["auto", "text2image", "image2image", "instructpix2pix"],
        help="Specify the task type. 'auto' detects based on --input-image. Use 'instructpix2pix' for that specific pipeline."
    )
    parser.add_argument(
        "--compiler",
        type=str,
        default=DEFAULT_COMPILER,
        choices=["none", "oneflow", "nexfort", "compile", "compile-max-autotune"],
        help="Compiler backend to use for optimization."
    )
    parser.add_argument(
        "--compiler-config",
        type=str,
        default=DEFAULT_COMPILER_CONFIG,
        help="JSON string for compiler configuration options."
    )
    parser.add_argument(
        "--run_multiple_resolutions",
        action="store_true", # Simplified to a flag
        help="Run tests with multiple common resolutions after the main generation."
    )
    parser.add_argument("--quantize", action="store_true", help="Enable quantization (currently requires --compiler nexfort).")
    parser.add_argument(
        "--quantize-config",
        type=str,
        default=DEFAULT_QUANTIZE_CONFIG,
         help="JSON string for quantization configuration."
    )
    parser.add_argument("--quant-submodules-config-path", type=str, default=None, help="Path to quantization submodules config file (for advanced nexfort quantization).")
    return parser.parse_args()

# --- Utility Functions ---

def load_pipe(
    pipeline_cls,
    model_name,
    variant=None,
    dtype=torch.float16,
    device="cuda",
    custom_pipeline=None,
    scheduler=None,
    lora=None,
    controlnet=None,
):
    """Loads the diffusion pipeline with optional components."""
    extra_kwargs = {}
    if custom_pipeline is not None:
        extra_kwargs["custom_pipeline"] = custom_pipeline
    if variant is not None:
        extra_kwargs["variant"] = variant
    if dtype is not None:
        extra_kwargs["torch_dtype"] = dtype

    # Handle ControlNet loading
    if controlnet is not None:
        from diffusers import ControlNetModel
        try:
            controlnet_model = ControlNetModel.from_pretrained(
                controlnet,
                torch_dtype=dtype,
            )
            extra_kwargs["controlnet"] = controlnet_model
            print(f"Successfully loaded ControlNet: {controlnet}")
        except Exception as e:
            print(f"Warning: Failed to load ControlNet '{controlnet}'. Error: {e}")
            print("Proceeding without ControlNet.")
            controlnet = None # Ensure controlnet is None if loading failed

    # Handle pre-quantized models (currently onediff specific)
    if os.path.exists(os.path.join(model_name, "calibrate_info.txt")):
         # Check if QuantPipeline is available before importing
        try:
            from onediff.quantization import QuantPipeline
            print(f"Found quantization info. Loading quantized model: {model_name}")
            pipe = QuantPipeline.from_quantized(pipeline_cls, model_name, **extra_kwargs)
        except ImportError:
            print("Warning: `onediff.quantization.QuantPipeline` not found. Install `onediff` for quantized model support.")
            print("Loading standard pipeline instead.")
            pipe = pipeline_cls.from_pretrained(model_name, **extra_kwargs)
        except Exception as e:
             print(f"Error loading quantized pipeline: {e}. Loading standard pipeline.")
             pipe = pipeline_cls.from_pretrained(model_name, **extra_kwargs)
    else:
        pipe = pipeline_cls.from_pretrained(model_name, **extra_kwargs)

    # Set Scheduler
    if scheduler is not None and scheduler.lower() != "none":
        try:
            scheduler_cls = getattr(importlib.import_module("diffusers"), scheduler)
            pipe.scheduler = scheduler_cls.from_config(pipe.scheduler.config)
            print(f"Using scheduler: {scheduler}")
        except (ImportError, AttributeError, Exception) as e:
            print(f"Warning: Failed to load or set scheduler '{scheduler}'. Using default. Error: {e}")

    # Load LoRA weights
    if lora is not None:
        try:
            print(f"Loading LoRA weights from: {lora}")
            pipe.load_lora_weights(lora)
            # Optionally fuse LoRA - check diffusers version compatibility if issues arise
            if hasattr(pipe, 'fuse_lora'):
                 print("Fusing LoRA weights.")
                 pipe.fuse_lora()
            else:
                 print("Warning: `pipe.fuse_lora()` not found. Skipping fusion (may require newer diffusers version).")
        except Exception as e:
            print(f"Warning: Failed to load or fuse LoRA weights from '{lora}'. Error: {e}")

    # Disable Safety Checker if present
    if hasattr(pipe, "safety_checker"):
        pipe.safety_checker = None
        print("Safety checker disabled.")

    # Move to device
    if device is not None:
        pipe.to(torch.device(device))
        print(f"Pipeline moved to device: {device}")

    return pipe

class IterationProfiler:
    """Profiles iterations per second during pipeline steps."""
    def __init__(self):
        self.begin = None
        self.end = None
        self.num_iterations = 0
        self.enabled = True # Flag to enable/disable profiling easily

    def get_iter_per_sec(self):
        if not self.enabled or self.begin is None or self.end is None or self.num_iterations == 0:
            return None
        try:
            self.end.synchronize() # Ensure timing is accurate
            dur = self.begin.elapsed_time(self.end) # Time in ms
            if dur == 0: return float('inf') # Avoid division by zero
            return self.num_iterations / dur * 1000.0 # Iterations per second
        except Exception as e:
            print(f"Warning: Error during iteration profiling: {e}")
            return None

    def callback_on_step_end(self, pipe, i, t, callback_kwargs={}):
        if not self.enabled:
            return callback_kwargs
        if torch.cuda.is_available():
            if self.begin is None:
                # Start timing on the first step
                event = torch.cuda.Event(enable_timing=True)
                event.record()
                self.begin = event
                self.num_iterations = 0 # Reset count at start
            else:
                # Record end event on subsequent steps
                event = torch.cuda.Event(enable_timing=True)
                event.record()
                self.end = event
                self.num_iterations += 1 # Increment count *after* the first step completes
        # Pass through callback_kwargs
        return callback_kwargs

    def reset(self):
        self.begin = None
        self.end = None
        self.num_iterations = 0

    def set_enabled(self, enabled=True):
        self.enabled = enabled
        if not enabled:
            self.reset()

# --- Throughput Analysis Functions ---

def calculate_inference_time_and_throughput(pipe, kwarg_inputs, n_steps, profiler):
    """Calculates inference time and throughput for a given number of steps."""
    kwarg_inputs_step = kwarg_inputs.copy()
    kwarg_inputs_step["num_inference_steps"] = n_steps

    # Reset and enable profiler for this run
    profiler.reset()
    profiler.set_enabled(True)

    start_time = time.time()
    # Use dummy generator for throughput test consistency if seed is None originally
    if kwarg_inputs_step.get("generator") is None:
         kwarg_inputs_step["generator"] = torch.Generator(device="cuda").manual_seed(DEFAULT_SEED or 0)

    _ = pipe(**kwarg_inputs_step) # Run inference
    torch.cuda.synchronize() # Ensure completion
    end_time = time.time()

    inference_time = end_time - start_time
    # Use profiler's calculation for it/s based on GPU events
    iter_per_sec = profiler.get_iter_per_sec()
    # Fallback: calculate based on wall time if profiler failed or steps < 2
    if iter_per_sec is None and inference_time > 0 and n_steps > 0 :
        steps_per_sec_wall = n_steps / inference_time
    else:
        steps_per_sec_wall = iter_per_sec if iter_per_sec is not None else 0

    # Disable profiler after use
    profiler.set_enabled(False)

    return inference_time, steps_per_sec_wall


def generate_data_and_fit_model(pipe, base_kwarg_inputs, steps_range, profiler):
    """Generates throughput data across a range of steps and fits a linear model."""
    print("\n--- Starting Throughput Analysis ---")
    data = {"steps": [], "inference_time": [], "throughput": []}
    height = base_kwarg_inputs.get('height', 512)
    width = base_kwarg_inputs.get('width', 512)

    for n_steps in steps_range:
        if n_steps <= 0: continue # Skip invalid step counts
        print(f"Testing {n_steps} steps...")
        try:
            inference_time, throughput = calculate_inference_time_and_throughput(
                pipe, base_kwarg_inputs, n_steps, profiler
            )
            data["steps"].append(n_steps)
            data["inference_time"].append(inference_time)
            # Store throughput (steps/sec)
            data["throughput"].append(throughput)
            print(
                f"  Steps: {n_steps}, Inference Time: {inference_time:.3f}s, Throughput: {throughput:.3f} steps/s"
            )
        except Exception as e:
            print(f"  Error during {n_steps} steps run: {e}")
            # Optionally break or continue
            # break
            continue
        # Short sleep to allow GPU cool-down if needed, can be removed
        # time.sleep(0.5)


    if not data["steps"] or len(data["steps"]) < 2 :
        print("Insufficient data points for throughput modeling.")
        return None, None

    df = pd.DataFrame(data)

    # Calculate Average Throughput from collected data
    # Exclude potential outliers (e.g., first few runs if warmup wasn't enough, or very low step counts)
    # Simple approach: exclude first point or use median/trimmed mean
    valid_throughputs = [t for t in data["throughput"] if t > 0 and np.isfinite(t)]
    if not valid_throughputs:
         print("No valid throughput measurements recorded.")
         average_throughput = 0
    else:
        average_throughput = np.mean(valid_throughputs) # Or np.median(valid_throughputs)
        print(f"\nAverage Measured Throughput: {average_throughput:.3f} steps/s")


    # Fit linear model: time = slope * steps + intercept
    # Requires at least 2 data points
    try:
        coefficients = np.polyfit(df["steps"], df["inference_time"], 1)
        slope = coefficients[0]
        intercept = coefficients[1]
        print(f"Linear Model Fit: Time = {slope:.4f} * Steps + {intercept:.4f}")

        # Estimate throughput based on the slope (time per step)
        if slope > 1e-9: # Avoid division by zero or near-zero
            throughput_from_slope = 1.0 / slope
            print(f"Throughput estimated from slope (ignoring base cost): {throughput_from_slope:.3f} steps/s")
        else:
            print("Slope is too small to estimate throughput reliably.")
            throughput_from_slope = None

    except np.linalg.LinAlgError as e:
        print(f"Could not fit linear model to data: {e}")
        coefficients = None
        throughput_from_slope = None


    print("--- Throughput Analysis Complete ---")
    return data, coefficients


def plot_data_and_model(data, coefficients):
    """Plots the inference time vs steps and the fitted linear model."""
    if data is None or not data["steps"]:
        print("No data to plot.")
        return

    plt.figure(figsize=(10, 6))
    plt.scatter(data["steps"], data["inference_time"], color="blue", label="Measured Data")

    if coefficients is not None and len(coefficients) == 2:
        steps_line = np.array(data["steps"])
        time_line = np.polyval(coefficients, steps_line)
        plt.plot(steps_line, time_line, color="red", label=f"Fit: Time = {coefficients[0]:.4f}*Steps + {coefficients[1]:.4f}")
        plt.legend()

    plt.title("Inference Time vs. Number of Steps")
    plt.xlabel("Number of Inference Steps")
    plt.ylabel("Inference Time (seconds)")
    plt.grid(True)
    plt.tight_layout()

    # Save or show the plot
    plot_filename = "throughput_analysis.png"
    try:
        plt.savefig(plot_filename)
        print(f"Throughput plot saved to {plot_filename}")
        # plt.show() # Uncomment to display interactively if not in a headless environment
    except Exception as e:
        print(f"Error saving/showing plot: {e}")

# --- Main Execution ---
def main():
    args = parse_args()

    # --- Determine Task and Pipeline Class ---
    effective_task = args.task
    if effective_task == "auto":
        if args.input_image is not None:
            effective_task = "image2image"
            print("Detected Image-to-Image task (input image provided).")
        else:
            effective_task = "text2image"
            print("Detected Text-to-Image task (no input image provided).")

    pipeline_cls = None
    if effective_task == "text2image":
        if args.deepcache:
             try:
                 # Requires onediffx.deep_cache module
                 from onediffx.deep_cache import StableDiffusionXLPipeline as PipelineForT2IDeepCache
                 from onediffx.deep_cache import StableDiffusionPipeline as SD15PipelineForT2IDeepCache # Example for SD 1.5
                 # Basic check if model name implies SDXL
                 if "xl" in args.model.lower():
                     pipeline_cls = PipelineForT2IDeepCache
                     print("Using DeepCache SDXL Text-to-Image pipeline.")
                 else:
                     # Assuming SD 1.5/2.1 for non-XL, adjust if needed
                     pipeline_cls = SD15PipelineForT2IDeepCache
                     print("Using DeepCache Stable Diffusion Text-to-Image pipeline.")

             except ImportError:
                 print("Warning: DeepCache pipelines not found in onediffx. Falling back to standard AutoPipeline.")
                 from diffusers import AutoPipelineForText2Image
                 pipeline_cls = AutoPipelineForText2Image
                 args.deepcache = False # Disable deepcache if import failed
        else:
            from diffusers import AutoPipelineForText2Image
            pipeline_cls = AutoPipelineForText2Image
            print("Using AutoPipelineForText2Image.")

    elif effective_task == "image2image":
        # For general I2I, AutoPipelineForImage2Image is suitable
        from diffusers import AutoPipelineForImage2Image
        pipeline_cls = AutoPipelineForImage2Image
        print("Using AutoPipelineForImage2Image.")

    elif effective_task == "instructpix2pix":
        if args.input_image is None:
            raise ValueError("--input-image is required for the 'instructpix2pix' task.")
        # Check if the specified model is likely InstructPix2Pix
        if "instruct-pix2pix" not in args.model.lower():
             print(f"Warning: Model '{args.model}' might not be an InstructPix2Pix model, but task='instructpix2pix' was specified.")
        from diffusers import StableDiffusionInstructPix2PixPipeline
        pipeline_cls = StableDiffusionInstructPix2PixPipeline
        print("Using StableDiffusionInstructPix2PixPipeline.")
        # InstructPix2Pix often uses specific default prompts
        if args.prompt == DEFAULT_PROMPT: # If user didn't override prompt
            args.prompt = "apply the instruction to the image" # More suitable default for instruct-pix2pix
            print(f"Using InstructPix2Pix specific default prompt: '{args.prompt}'")

    else:
        # This case should not be reached due to argparse choices
         raise ValueError(f"Invalid task specified: {args.task}")

    if pipeline_cls is None:
         raise RuntimeError("Could not determine the pipeline class. Check task and model.")


    # --- Load Pipeline ---
    print(f"\nLoading model: {args.model}")
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32 # Use float16 on CUDA by default
    pipe = load_pipe(
        pipeline_cls,
        args.model,
        variant=args.variant,
        custom_pipeline=args.custom_pipeline,
        scheduler=args.scheduler,
        lora=args.lora,
        controlnet=args.controlnet,
        dtype=dtype,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    # --- Determine Optimal Height/Width ---
    # Use provided H/W if set, otherwise try to infer from model
    height = args.height
    width = args.width
    if height is None or width is None:
        try:
            model_height = pipe.unet.config.sample_size * pipe.vae_scale_factor
            model_width = pipe.unet.config.sample_size * pipe.vae_scale_factor
            if height is None: height = model_height
            if width is None: width = model_width
            print(f"Auto-detected resolution: {height}x{width}")
        except AttributeError:
            # Fallback if detection fails (e.g., non-standard pipeline structure)
            fallback_res = 512 if "xl" not in args.model.lower() else 1024
            if height is None: height = fallback_res
            if width is None: width = fallback_res
            print(f"Warning: Could not auto-detect resolution. Using default: {height}x{width}")

    # Ensure height and width are multiples of VAE scale factor (usually 8)
    vae_scale_factor = getattr(pipe, "vae_scale_factor", 8)
    height = (height // vae_scale_factor) * vae_scale_factor
    width = (width // vae_scale_factor) * vae_scale_factor
    if args.height != height or args.width != width:
         print(f"Adjusted resolution to be multiples of {vae_scale_factor}: {height}x{width}")


    # --- Apply Compiler/Quantization ---
    compiled = False
    if args.compiler != "none" and torch.cuda.is_available():
        print(f"\nApplying compiler: {args.compiler}")
        if args.compiler == "oneflow":
            # Requires oneflow and onediff to be installed
            try:
                import oneflow # Check if oneflow is installed
                pipe = compile_pipe(pipe, backend="oneflow") # Assumes compile_pipe handles backend selection
                compiled = True
                print("Oneflow backend via compile_pipe is active.")
            except ImportError:
                print("Warning: OneFlow not installed. Skipping OneFlow compilation.")
            except Exception as e:
                print(f"Error during OneFlow compilation: {e}. Proceeding without compilation.")

        elif args.compiler == "nexfort":
            # Requires nexfort, torchao, onediffx
            try:
                quantize_options = {}
                if args.quantize:
                    print("Applying Nexfort quantization...")
                    if args.quantize_config:
                        try:
                            quantize_options = json.loads(args.quantize_config)
                            print(f"Using custom quantize config: {quantize_options}")
                        except json.JSONDecodeError:
                            print(f"Warning: Invalid JSON in --quantize-config. Using default.")
                            quantize_options = {"quant_type": "fp8_e4m3_e4m3_dynamic"} # Default FP8
                    else:
                         quantize_options = {"quant_type": "fp8_e4m3_e4m3_dynamic"} # Default FP8
                         print(f"Using default quantize config: {quantize_options}")

                    if args.quant_submodules_config_path:
                         print(f"Using quant submodules config: {args.quant_submodules_config_path}")
                         pipe = quantize_pipe(
                             pipe,
                             quant_submodules_config_path=args.quant_submodules_config_path,
                             ignores=[], # Example: Add submodules to ignore if needed
                              **quantize_options
                         )
                    else:
                         pipe = quantize_pipe(pipe, ignores=[], **quantize_options)
                    print("Quantization applied.")


                compiler_options = {}
                if args.compiler_config:
                    try:
                        compiler_options = json.loads(args.compiler_config)
                        print(f"Using custom compiler config: {compiler_options}")
                    except json.JSONDecodeError:
                         print(f"Warning: Invalid JSON in --compiler-config. Using default.")
                         # Safe default options string
                         compiler_options = {"mode": "max-optimize:max-autotune:freezing", "memory_format": "channels_last"}
                else:
                    # Safe default options string
                    compiler_options = {"mode": "max-optimize:max-autotune:freezing", "memory_format": "channels_last"}
                    print(f"Using default compiler config: {compiler_options}")

                # Apply compilation
                pipe = compile_pipe(
                    pipe,
                    backend="nexfort",
                    options=compiler_options,
                    fuse_qkv_projections=True # Generally safe and beneficial
                )
                compiled = True
                print("Nexfort backend is active.")

            except ImportError as e:
                 print(f"Warning: Missing dependencies for nexfort ({e}). Skipping nexfort compilation.")
            except Exception as e:
                 print(f"Error during Nexfort setup: {e}. Proceeding without nexfort.")


        elif args.compiler in ("compile", "compile-max-autotune"):
             # Uses torch.compile
            mode = "max-autotune" if args.compiler == "compile-max-autotune" else None
            print(f"Applying torch.compile (mode: {mode or 'default'})...")
            # Compile relevant components
            compiled_components = []
            for component_name in ["unet", "vae", "transformer", "controlnet"]:
                 if hasattr(pipe, component_name) and getattr(pipe, component_name) is not None:
                     try:
                         print(f"Compiling {component_name}...")
                         setattr(pipe, component_name, torch.compile(getattr(pipe, component_name), mode=mode))
                         compiled_components.append(component_name)
                     except Exception as e:
                         print(f"Warning: Failed to compile {component_name}. Error: {e}")

            if compiled_components:
                 print(f"Successfully compiled: {', '.join(compiled_components)}")
                 compiled = True
            else:
                 print("No components were compiled with torch.compile.")

        else:
             # Should not happen due to argparse choices
             print(f"Warning: Unknown compiler '{args.compiler}' requested. Running in eager mode.")
    elif args.compiler != "none":
        print("CUDA not available, skipping compilation.")


    # --- Load Images (Input and Control) ---
    input_image = None
    if args.input_image:
        print(f"Loading input image: {args.input_image}")
        try:
            input_image = load_image(args.input_image)
            input_image = input_image.resize((width, height), Image.LANCZOS)
            print(f"Input image resized to {width}x{height}")
        except Exception as e:
            print(f"Error loading or resizing input image: {e}. Cannot perform Image-to-Image task.")
            return # Exit if I2I is required but image loading fails

    control_image = None
    if args.control_image:
        if args.controlnet is None:
             print("Warning: --control-image provided but no --controlnet model specified. Control image will be ignored.")
        else:
            print(f"Loading control image: {args.control_image}")
            try:
                control_image = load_image(args.control_image)
                control_image = control_image.resize((width, height), Image.LANCZOS)
                print(f"Control image resized to {width}x{height}")
            except Exception as e:
                 print(f"Warning: Error loading or resizing control image: {e}. Proceeding without control image.")
                 control_image = None # Ensure it's None if loading fails
    elif args.controlnet is not None and input_image is not None:
        # If controlnet is specified but no specific control image, use the input image as control
        print("Using input image as control image for ControlNet.")
        control_image = input_image
    elif args.controlnet is not None:
        # ControlNet specified but no control image and no input image (T2I mode)
        # Generate a dummy control image (e.g., blank) or raise error?
        # Let's create a blank white image as a placeholder
        print("Warning: ControlNet specified but no --control-image or --input-image provided.")
        print(f"Creating a blank white control image ({width}x{height}).")
        control_image = Image.new("RGB", (width, height), (255, 255, 255))
        # Or could raise error: raise ValueError("ControlNet requires --control-image or --input-image.")


    # --- Prepare Keyword Arguments for Pipeline Call ---
    def get_kwarg_inputs(current_args, current_height, current_width, current_input_image, current_control_image):
        kwarg_inputs = dict(
            prompt=current_args.prompt,
            negative_prompt=current_args.negative_prompt,
            height=current_height,
            width=current_width,
            num_images_per_prompt=current_args.batch,
            num_inference_steps=current_args.steps, # Ensure steps are included
            generator=(
                None
                if current_args.seed is None
                else torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu").manual_seed(current_args.seed)
            ),
        )

        # Add image for I2I tasks
        if effective_task in ["image2image", "instructpix2pix"] and current_input_image is not None:
            kwarg_inputs["image"] = current_input_image
        elif effective_task in ["image2image", "instructpix2pix"]:
             # Should have been caught earlier, but double-check
             raise RuntimeError(f"Input image is required for task '{effective_task}' but is missing.")


        # Add control image if available and ControlNet is loaded
        if current_control_image is not None and hasattr(pipe, 'controlnet') and pipe.controlnet is not None:
             # Some pipelines expect 'control_image', others 'image' if it's the primary input
             # Check signature - this is complex, maybe rely on AutoPipeline or specific pipeline needs
             # Simple approach: Add 'control_image' if ControlNet is present.
             # If it conflicts with 'image', the specific pipeline should handle it or error out.
             kwarg_inputs["control_image"] = current_control_image
             # For T2I + ControlNet, sometimes 'image' needs to be the control image
             if effective_task == "text2image" and "image" not in kwarg_inputs:
                  kwarg_inputs["image"] = current_control_image # Pass control image as 'image' for T2I ControlNet
                  print("Passing control image as 'image' argument for T2I + ControlNet task.")


        # Add DeepCache arguments if enabled
        if current_args.deepcache and effective_task == "text2image": # Currently example DeepCache pipeline is T2I
            # Check if the pipeline actually supports these args (might need more robust check)
            sig = inspect.signature(pipe.__call__)
            if "cache_interval" in sig.parameters:
                kwarg_inputs["cache_interval"] = current_args.cache_interval
                kwarg_inputs["cache_layer_id"] = current_args.cache_layer_id
                kwarg_inputs["cache_block_id"] = current_args.cache_block_id
            else:
                print("Warning: --deepcache specified, but pipeline does not seem to support cache arguments. Disabling.")
                args.deepcache = False # Disable if not supported

        # Add extra keyword arguments from JSON string
        if current_args.extra_call_kwargs:
            try:
                extra_kwargs = json.loads(current_args.extra_call_kwargs)
                # Filter out args already handled explicitly to avoid conflicts
                keys_to_remove = {"prompt", "negative_prompt", "height", "width", "num_images_per_prompt", "generator", "num_inference_steps", "image", "control_image", "cache_interval", "cache_layer_id", "cache_block_id", "callback_on_step_end", "callback"}
                filtered_extra_kwargs = {k: v for k, v in extra_kwargs.items() if k not in keys_to_remove}

                # Check for potential conflicts (e.g., guidance_scale vs EtaDDIM) - let diffusers handle it mostly
                print(f"Adding extra call arguments: {filtered_extra_kwargs}")
                kwarg_inputs.update(filtered_extra_kwargs)
            except json.JSONDecodeError as e:
                print(f"Warning: Invalid JSON in --extra-call-kwargs: {e}. Ignoring extra args.")

        return kwarg_inputs

    # --- Warmup Runs ---
    if args.warmups > 0:
        print("\n=======================================")
        print(f"Begin warmup ({args.warmups} runs)...")
        # Use a temporary profiler for warmup that's disabled
        warmup_profiler = IterationProfiler()
        warmup_profiler.set_enabled(False)
        warmup_kwarg_inputs = get_kwarg_inputs(args, height, width, input_image, control_image)
        # Add dummy callback if needed by signature, but disabled profiler won't use it
        sig = inspect.signature(pipe.__call__)
        if "callback_on_step_end" in sig.parameters:
             warmup_kwarg_inputs["callback_on_step_end"] = warmup_profiler.callback_on_step_end
        elif "callback" in sig.parameters: # Older diffusers convention
             warmup_kwarg_inputs["callback"] = warmup_profiler.callback_on_step_end

        start_warmup_time = time.time()
        for i in range(args.warmups):
            # Ensure generator is reset/new for each warmup if seed is None
            current_seed = args.seed if args.seed is not None else int(time.time()) + i
            warmup_kwarg_inputs["generator"] = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu").manual_seed(current_seed)
            _ = pipe(**warmup_kwarg_inputs)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        end_warmup_time = time.time()
        print("End warmup")
        print(f"Warmup time: {end_warmup_time - start_warmup_time:.3f}s")
        print("=======================================")
        del warmup_profiler # Clean up

    # --- Timed Inference Run ---
    print("\n=======================================")
    print("Begin timed inference run...")
    iter_profiler = IterationProfiler()
    # Ensure profiler is enabled for the main run
    iter_profiler.set_enabled(torch.cuda.is_available()) # Only enable if CUDA is available for timing events

    kwarg_inputs = get_kwarg_inputs(args, height, width, input_image, control_image)

    # Add the profiling callback
    sig = inspect.signature(pipe.__call__)
    if "callback_on_step_end" in sig.parameters:
        kwarg_inputs["callback_on_step_end"] = iter_profiler.callback_on_step_end
        print("Iteration profiler attached via callback_on_step_end.")
    elif "callback" in sig.parameters: # Older diffusers convention
        kwarg_inputs["callback"] = iter_profiler.callback_on_step_end
        print("Iteration profiler attached via callback.")
    else:
        iter_profiler.set_enabled(False) # Disable profiler if no callback mechanism found
        print("Warning: Pipeline does not support step callbacks. Iteration profiling disabled.")


    # Clear CUDA cache before timed run (optional, might help consistency)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        initial_mem = torch.cuda.max_memory_allocated() / (1024**3) # GiB

    start_time = time.time()
    output_images = pipe(**kwarg_inputs).images
    if torch.cuda.is_available():
        torch.cuda.synchronize() # Wait for GPU to finish
    end_time = time.time()

    inference_time = end_time - start_time
    print("Inference complete.")
    print("=======================================")
    print(f"Task Type: {effective_task.upper()}")
    print(f"Model: {args.model}")
    if compiled: print(f"Compiler: {args.compiler}")
    if args.quantize: print("Quantization: Enabled (Nexfort)")
    print(f"Resolution: {height}x{width}")
    print(f"Steps: {args.steps}")
    print(f"Inference Time (Wall Clock): {inference_time:.3f}s")

    # Report Iterations Per Second from profiler
    iter_per_sec = iter_profiler.get_iter_per_sec()
    if iter_per_sec is not None:
        print(f"Iterations Per Second (GPU Profiled): {iter_per_sec:.3f}")
    elif iter_profiler.enabled and args.steps > 1:
         print("Iterations Per Second (GPU Profiled): N/A (profiling error or too few steps)")
    elif inference_time > 0 and args.steps > 0:
        # Fallback to wall clock steps/sec if profiler not available/failed
        wall_steps_per_sec = args.steps / inference_time
        print(f"Steps Per Second (Wall Clock): {wall_steps_per_sec:.3f}")


    # Report Memory Usage
    if torch.cuda.is_available():
        # Use torch.cuda.max_memory_allocated() which tracks peak usage
        cuda_mem_after_used = torch.cuda.max_memory_allocated() / (1024**3) # GiB
        # Reset peak stats for next potential runs if needed
        torch.cuda.reset_peak_memory_stats()
        print(f"Max CUDA Memory Used: {cuda_mem_after_used:.3f} GiB")
    # elif args.compiler == "oneflow": # Specific check for oneflow if needed
    #     try:
    #         import oneflow as flow
    #         # Note: OneFlow's memory reporting might differ
    #         cuda_mem_after_used = flow._oneflow_internal.GetCUDAMemoryUsed() / 1024 # KiB? Check unit
    #         print(f"Max used OneFlow CUDA memory : {cuda_mem_after_used:.3f} Units (check OneFlow docs for unit)")
    #     except ImportError:
    #         pass # Oneflow not installed
    print("=======================================")

    # --- Save Output Image ---
    if args.output_image:
        try:
            output_images[0].save(args.output_image)
            print(f"Output image saved to: {args.output_image}")
        except IndexError:
            print("Error: No images generated.")
        except Exception as e:
            print(f"Error saving output image to {args.output_image}: {e}")
    else:
        print("No --output-image path specified. Image not saved.")

    # --- Optional: Run Multiple Resolutions Test ---
    if args.run_multiple_resolutions:
        print("\n--- Testing Multiple Resolutions ---")
        sizes = [1024, 768, 512, 256] # Example sizes
        base_kwarg_inputs = get_kwarg_inputs(args, height, width, input_image, control_image)
        # Remove callbacks for these runs if they were added
        base_kwarg_inputs.pop("callback_on_step_end", None)
        base_kwarg_inputs.pop("callback", None)

        for h in sizes:
            for w in sizes:
                 # Skip if same as original run
                 if h == height and w == width: continue

                 # Adjust resolution, ensuring it's valid (multiple of 8)
                 h_test = (h // vae_scale_factor) * vae_scale_factor
                 w_test = (w // vae_scale_factor) * vae_scale_factor
                 if h_test == 0 or w_test == 0: continue # Skip invalid zero sizes

                 print(f"Running at resolution: {h_test}x{w_test}")
                 current_kwarg_inputs = base_kwarg_inputs.copy()
                 current_kwarg_inputs["height"] = h_test
                 current_kwarg_inputs["width"] = w_test

                 # Need to resize input/control images if they exist for I2I
                 current_input_image_test = None
                 if input_image:
                      try:
                          current_input_image_test = input_image.resize((w_test, h_test), Image.LANCZOS)
                          if effective_task in ["image2image", "instructpix2pix"]:
                               current_kwarg_inputs["image"] = current_input_image_test
                      except Exception as e:
                          print(f"  Warn: Failed to resize input image for {h_test}x{w_test}. Skipping.")
                          continue

                 current_control_image_test = None
                 if control_image:
                      try:
                          current_control_image_test = control_image.resize((w_test, h_test), Image.LANCZOS)
                          if "control_image" in current_kwarg_inputs: # Check if key exists
                               current_kwarg_inputs["control_image"] = current_control_image_test
                          if effective_task == "text2image" and args.controlnet: # T2I+ControlNet case
                              current_kwarg_inputs["image"] = current_control_image_test
                      except Exception as e:
                           print(f"  Warn: Failed to resize control image for {h_test}x{w_test}. Skipping.")
                           continue


                 # Reset generator for consistency if seed is None
                 if args.seed is None:
                     current_kwarg_inputs["generator"] = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu").manual_seed(DEFAULT_SEED or 0) # Use default seed for test runs


                 try:
                     start_res_time = time.time()
                     _ = pipe(**current_kwarg_inputs).images
                     if torch.cuda.is_available(): torch.cuda.synchronize()
                     end_res_time = time.time()
                     print(f"  Inference time: {end_res_time - start_res_time:.3f} seconds")
                 except Exception as e:
                     print(f"  Error during {h_test}x{w_test} run: {e}")
                 # Optional: small delay
                 # time.sleep(0.5)
        print("--- Multi-resolution Testing Complete ---")


    # --- Optional: Throughput Analysis ---
    if args.throughput:
        # Use a range starting from a few steps up to maybe slightly more than default
        steps_range = range(5, max(55, args.steps + 15), 5) # e.g., 5, 10, 15... up to 50 or more
        base_kwarg_inputs = get_kwarg_inputs(args, height, width, input_image, control_image)
        # Remove callbacks for throughput runs as we use a dedicated profiler inside
        base_kwarg_inputs.pop("callback_on_step_end", None)
        base_kwarg_inputs.pop("callback", None)

        throughput_data, throughput_coeffs = generate_data_and_fit_model(
            pipe, base_kwarg_inputs, steps_range, iter_profiler # Reuse main profiler object
            )
        if throughput_data:
             plot_data_and_model(throughput_data, throughput_coeffs)

if __name__ == "__main__":
    main()

Writing generate.py


In [4]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DreamDoodle - AI Image Generation</title>
    <!-- Tailwind CSS via CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <!-- Font Awesome via CDN -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');

        body {
            font-family: 'Poppins', sans-serif;
            background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
            min-height: 100vh;
            display: flex; /* Use flexbox for footer */
            flex-direction: column; /* Stack elements vertically */
        }

        .gradient-text {
            background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            background-clip: text;
            color: transparent;
        }

        .image-preview {
            transition: all 0.3s ease;
            box-shadow: 0 10px 15px -3px rgba(0, 0, 0, 0.1), 0 4px 6px -2px rgba(0, 0, 0, 0.05);
        }

        .image-preview:hover {
            transform: translateY(-2px);
            box-shadow: 0 20px 25px -5px rgba(0, 0, 0, 0.1), 0 10px 10px -5px rgba(0, 0, 0, 0.04);
        }

        .upload-area {
            border: 2px dashed #cbd5e0;
            transition: all 0.3s ease;
        }

        .upload-area:hover {
            border-color: #667eea;
            background-color: rgba(102, 126, 234, 0.05);
        }

        .upload-area.dragover {
            border-color: #667eea;
            background-color: rgba(102, 126, 234, 0.1);
        }

        .generated-image {
            animation: fadeIn 0.5s ease-in-out;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: scale(0.95); }
            to { opacity: 1; transform: scale(1); }
        }

        /* --- Styling for Flask Error Messages --- */
        .flask-error {
             background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb;
             margin-top: 20px; padding: 15px; border-radius: 8px; text-align: center;
        }
        .flask-error pre {
             white-space: pre-wrap; word-wrap: break-word; text-align: left; font-family: monospace;
             font-size: 0.9em; max-height: 200px; overflow-y: auto; background-color: #f1f1f1;
             padding: 10px; border-radius: 3px; margin-top: 10px;
         }

        /* --- Styling for Client-Side Toast --- */
        .toast-container {
            position: fixed;
            bottom: 1rem;
            right: 1rem;
            z-index: 50;
        }
        .toast {
            animation: slideIn 0.3s ease-out, fadeOut 0.5s ease-out 2.5s forwards;
            display: flex;
            align-items: center;
            padding: 0.75rem 1rem; /* py-3 px-4 */
            border-radius: 0.5rem; /* rounded-lg */
            box-shadow: 0 10px 15px -3px rgba(0, 0, 0, 0.1), 0 4px 6px -2px rgba(0, 0, 0, 0.05); /* shadow-lg */
            background-color: #ef4444; /* bg-red-500 */
            color: white;
        }
         .toast i { margin-right: 0.5rem; /* mr-2 */}

        @keyframes slideIn {
            from { transform: translateY(20px); opacity: 0; }
            to { transform: translateY(0); opacity: 1; }
        }
        @keyframes fadeOut {
            from { opacity: 1; }
            to { opacity: 0; }
        }
    </style>
</head>
<body class="min-h-screen flex flex-col">
    <!-- Header -->
    <header class="bg-white shadow-sm py-4">
        <div class="container mx-auto px-4">
            <div class="flex justify-between items-center">
                <h1 class="text-3xl font-bold gradient-text">DreamDoodle</h1>
                <div class="flex items-center space-x-4">
                    <!-- Sign in button is just decorative for now -->
                    <button class="px-4 py-2 bg-gradient-to-r from-purple-500 to-indigo-600 text-white rounded-lg hover:opacity-90 transition">
                        <i class="fas fa-sign-in-alt mr-2"></i>Sign In
                    </button>
                </div>
            </div>
        </div>
    </header>

    <!-- Main Content -->
    <main class="flex-grow container mx-auto px-4 py-8">
        <div class="max-w-4xl mx-auto">
            <div class="text-center mb-12">
                <h2 class="text-4xl font-bold text-gray-800 mb-3">Transform Your Imagination into Reality</h2>
                <p class="text-xl text-gray-600">Create stunning images with AI. Describe your vision or upload an image to modify.</p>
            </div>

            <!-- Display Flask Error Messages -->
            {% if error %}
                <div class="flask-error">
                    <strong>Error:</strong> {{ error }}
                    {% if error_details %}
                        <pre>{{ error_details }}</pre>
                    {% endif %}
                </div>
            {% endif %}

            <!-- Generation Form -->
            <!-- Ensure form submits via POST and handles file uploads -->
            <form id="generate-form" method="post" enctype="multipart/form-data" action="{{ url_for('index') }}">
                <div class="bg-white rounded-xl shadow-lg p-6 mb-8">
                    <div class="mb-6">
                        <label for="prompt" class="block text-lg font-medium text-gray-700 mb-2">
                            <i class="fas fa-magic mr-2 text-indigo-500"></i>Describe your dream image
                        </label>
                        <textarea
                            id="prompt"
                            name="prompt"  {# <<< Ensure name attribute is set #}
                            rows="4"
                            class="w-full px-4 py-3 border border-gray-300 rounded-lg focus:ring-2 focus:ring-indigo-500 focus:border-indigo-500 transition"
                            placeholder="A majestic lion standing on a cliff at sunset, hyper-realistic, 8K resolution..."
                            required {# <<< Add required attribute for basic HTML validation #}
                        >{{ prompt or '' }}</textarea> {# <<< Repopulate prompt #}
                    </div>

                    <!-- Image Upload -->
                    <div class="mb-6">
                        <label class="block text-lg font-medium text-gray-700 mb-2">
                            <i class="fas fa-image mr-2 text-indigo-500"></i>Upload an image (optional)
                        </label>
                        <div
                            id="uploadArea"
                            class="upload-area relative rounded-lg p-8 text-center cursor-pointer"
                        >
                            <!-- Ensure name attribute is set -->
                            <input type="file" id="imageUpload" name="input_image" class="hidden" accept="image/*">
                            <div id="uploadContent" class="space-y-2">
                                <i class="fas fa-cloud-upload-alt text-4xl text-indigo-400"></i>
                                <p class="text-gray-600">Drag & drop your image here or click to browse</p>
                                <p class="text-sm text-gray-500">Supports JPG, PNG, WEBP (Max 5MB recommended)</p>
                            </div>
                            <div id="imagePreviewContainer" class="hidden">
                                <div class="relative inline-block">
                                    <img id="imagePreview" src="#" alt="Preview" class="image-preview max-h-64 rounded-lg">
                                    <button type="button" id="removeImage" class="absolute top-2 right-2 bg-red-500 text-white rounded-full p-1 hover:bg-red-600 transition">
                                        <i class="fas fa-times"></i>
                                    </button>
                                </div>
                            </div>
                        </div>
                    </div>

                    <!-- Generate Button -->
                    <div class="text-center">
                        <button
                            id="generateBtn"
                            type="submit" {# <<< Ensure type is submit #}
                            class="px-8 py-3 bg-gradient-to-r from-purple-600 to-indigo-700 text-white text-lg font-semibold rounded-lg hover:opacity-90 transition transform hover:scale-105 shadow-lg disabled:opacity-50 disabled:cursor-not-allowed"
                        >
                            <i class="fas fa-sparkles mr-2"></i>Generate Image
                        </button>
                         <!-- Loading Spinner (hidden initially) -->
                         <div id="loadingSpinner" class="hidden text-center mt-4">
                             <i class="fas fa-spinner fa-spin text-3xl text-indigo-600"></i>
                             <p class="text-gray-600 mt-2">Generating... please wait (this can take a minute or two).</p>
                         </div>
                    </div>
                </div>
            </form>

            <!-- Generated Image Display - Controlled by Flask Template Logic -->
            {% if image_url %}
            <div id="resultSection" class="bg-white rounded-xl shadow-lg p-6">
                <h3 class="text-2xl font-semibold text-gray-800 mb-4 text-center">
                    <i class="fas fa-image mr-2 text-indigo-500"></i>Your Generated Image
                </h3>
                 <div class="text-center text-gray-600 mb-4">
                     <p><strong>Prompt:</strong> {{ prompt }}</p>
                     {% if input_image_filename %}
                      <p><strong>Based on input:</strong> {{ input_image_filename }}</p>
                     {% endif %}
                 </div>
                <div class="flex justify-center">
                    <div id="generatedImageContainer" class="relative">
                        <!-- Inject image URL from Flask -->
                        <img id="generatedImage" src="{{ image_url }}?t={{ timestamp }}" alt="Generated Image" class="generated-image max-w-full rounded-lg shadow-md">
                        <div class="mt-4 flex justify-center space-x-4">
                            <button id="downloadBtn" type="button" class="download-btn px-4 py-2 bg-green-500 text-white rounded-lg hover:bg-green-600 transition">
                                <i class="fas fa-download mr-2"></i>Download
                            </button>
                            <button id="regenerateBtn" type="button" class="regenerate-btn px-4 py-2 bg-indigo-500 text-white rounded-lg hover:bg-indigo-600 transition">
                                <i class="fas fa-sync-alt mr-2"></i>Regenerate
                            </button>
                        </div>
                    </div>
                </div>
            </div>
            {% endif %} {# End if image_url #}

        </div>
    </main>

    <!-- Footer -->
    <footer class="bg-gray-800 text-white py-6 mt-auto"> {# Added mt-auto for sticky footer effect #}
        <div class="container mx-auto px-4">
            <div class="flex flex-col md:flex-row justify-between items-center">
                <div class="mb-4 md:mb-0">
                    <h2 class="text-2xl font-bold gradient-text">DreamDoodle</h2>
                    <p class="text-gray-400">Powered by AI magic</p>
                </div>
                <div class="flex space-x-6">
                    <a href="#" class="hover:text-indigo-300 transition">Terms</a>
                    <a href="#" class="hover:text-indigo-300 transition">Privacy</a>
                    <a href="#" class="hover:text-indigo-300 transition">Contact</a>
                </div>
            </div>
            <div class="mt-6 text-center text-gray-400 text-sm">
                © 2024 DreamDoodle. All rights reserved. {# Updated year #}
            </div>
        </div>
    </footer>

    <!-- Toast Notification Container -->
    <div id="toastContainer" class="toast-container">
         <!-- Toast messages will be added here by JS -->
    </div>

    <script>
        document.addEventListener('DOMContentLoaded', function() {
            // DOM Elements
            const uploadArea = document.getElementById('uploadArea');
            const imageUpload = document.getElementById('imageUpload');
            const uploadContent = document.getElementById('uploadContent');
            const imagePreviewContainer = document.getElementById('imagePreviewContainer');
            const imagePreview = document.getElementById('imagePreview');
            const removeImage = document.getElementById('removeImage');
            const promptInput = document.getElementById('prompt');
            const generateBtn = document.getElementById('generateBtn');
            const generateForm = document.getElementById('generate-form'); // Get the form itself
            const loadingSpinner = document.getElementById('loadingSpinner');
            const toastContainer = document.getElementById('toastContainer');

            // Result section elements (might not exist on initial load)
            const resultSection = document.getElementById('resultSection');
            const downloadBtn = document.getElementById('downloadBtn');
            const regenerateBtn = document.getElementById('regenerateBtn');
            const generatedImage = document.getElementById('generatedImage'); // Needed for download

            let uploadedFile = null; // Store the File object for resubmission

            // --- Event Listeners ---

            // Handle drag and drop
            ['dragenter', 'dragover', 'dragleave', 'drop'].forEach(eventName => {
                uploadArea.addEventListener(eventName, preventDefaults, false);
            });

            function preventDefaults(e) {
                e.preventDefault();
                e.stopPropagation();
            }

            ['dragenter', 'dragover'].forEach(eventName => {
                uploadArea.addEventListener(eventName, () => uploadArea.classList.add('dragover'), false);
            });

            ['dragleave', 'drop'].forEach(eventName => {
                uploadArea.addEventListener(eventName, () => uploadArea.classList.remove('dragover'), false);
            });

            uploadArea.addEventListener('drop', handleDrop, false);
            uploadArea.addEventListener('click', () => imageUpload.click()); // Trigger file input click

            function handleDrop(e) {
                const dt = e.dataTransfer;
                handleFiles(dt.files);
            }

            // Handle file selection via click
            imageUpload.addEventListener('change', function() {
                handleFiles(this.files);
            });

            function handleFiles(files) {
                if (files.length > 0) {
                    const file = files[0];
                    if (file.type.startsWith('image/')) {
                        // Simple size check (adjust limit if needed)
                        if (file.size > 10 * 1024 * 1024) { // 10 MB limit example
                            showToast('Image size should be less than 10MB');
                            resetUpload();
                            return;
                        }

                        uploadedFile = file; // Store the actual file object

                        const reader = new FileReader();
                        reader.onload = function(e) {
                            imagePreview.src = e.target.result; // Use reader result for preview
                            uploadContent.classList.add('hidden');
                            imagePreviewContainer.classList.remove('hidden');
                        }
                        reader.readAsDataURL(file);
                    } else {
                        showToast('Please upload a valid image file (JPG, PNG, WEBP).');
                        resetUpload();
                    }
                }
            }

             // Remove uploaded image preview
             removeImage.addEventListener('click', function(e) {
                 e.stopPropagation(); // Prevent triggering upload area click
                 resetUpload();
             });

             function resetUpload() {
                 uploadedFile = null;
                 imagePreview.src = '#';
                 imageUpload.value = ''; // Clear the file input
                 uploadContent.classList.remove('hidden');
                 imagePreviewContainer.classList.add('hidden');
             }

            // Handle Form Submission
            generateForm.addEventListener('submit', function(event) {
                const promptValue = promptInput.value.trim();

                // Client-side validation
                if (!promptValue) {
                    showToast('Please enter a prompt to generate an image.');
                    event.preventDefault(); // Stop submission
                    return;
                }
                // The requirement "cannot submit with only image" is handled by requiring the prompt.

                // Show loading state AFTER validation passes
                generateBtn.disabled = true;
                generateBtn.innerHTML = '<i class="fas fa-spinner fa-spin mr-2"></i>Generating...';
                loadingSpinner.classList.remove('hidden');

                // Allow the form to submit naturally to the Flask backend
            });


            // Add functionality to Download button (if it exists on the page)
            if (downloadBtn && generatedImage) {
                 downloadBtn.addEventListener('click', function() {
                     const imageUrl = generatedImage.src.split('?')[0]; // Get URL without timestamp
                     const filename = imageUrl.substring(imageUrl.lastIndexOf('/') + 1) || 'dreamdoodle_image.png';

                     // Create temporary link to trigger download
                     const link = document.createElement('a');
                     link.href = imageUrl;
                     link.download = filename;
                     document.body.appendChild(link);
                     link.click();
                     document.body.removeChild(link);
                 });
            }

             // Add functionality to Regenerate button (if it exists)
             if (regenerateBtn) {
                 regenerateBtn.addEventListener('click', function() {
                     // Re-enable button temporarily if needed, show spinner, and resubmit
                     // Note: Resubmitting the form might not re-upload the same file easily across all browsers.
                     // A more robust way might involve storing form data and resubmitting via JS fetch,
                     // but for simplicity, we'll just trigger the button click again, assuming the user
                     // hasn't changed the prompt/image significantly.
                     console.log("Regenerate clicked");
                     // Ensure prompt is still populated from the template rendering
                     if (!promptInput.value.trim()){
                         showToast("Cannot regenerate without a prompt.");
                         return;
                     }

                      // Simulate clicking the main generate button again
                      generateBtn.disabled = false; // Re-enable briefly if needed
                      generateBtn.click(); // This will trigger the form submit listener again
                 });
             }


            // Show toast message function
            function showToast(message, type = 'error') {
                 const toast = document.createElement('div');
                 toast.className = 'toast mb-2'; // Add margin between toasts if multiple show quickly
                 let iconClass = 'fa-exclamation-circle';
                 let bgColor = 'bg-red-500'; // Default error

                 // Add more types if needed (e.g., success, info)
                 // if (type === 'success') { iconClass = 'fa-check-circle'; bgColor = 'bg-green-500'; }
                 // if (type === 'info') { iconClass = 'fa-info-circle'; bgColor = 'bg-blue-500'; }

                 toast.classList.add(bgColor); // Apply background based on type

                 toast.innerHTML = `
                     <i class="fas ${iconClass} mr-2"></i>
                     <span>${message}</span>
                 `;
                 toastContainer.appendChild(toast);

                 // Automatically remove the toast after ~3 seconds
                 setTimeout(() => {
                     toast.remove();
                 }, 3000);
            }
        });
    </script>
</body>
</html>

Writing templates/index.html


In [5]:
%%writefile app.py

import os
import subprocess
import uuid # For unique filenames
import time
from flask import Flask, request, render_template, redirect, url_for, flash, send_from_directory
from werkzeug.utils import secure_filename # For safe filename handling

# --- Configuration ---
# Use absolute paths in Colab environment
APP_ROOT = '/content/DreamDoodleWebApp' # Base directory
UPLOAD_FOLDER = os.path.join(APP_ROOT, 'static', 'uploads')
RESULT_FOLDER = os.path.join(APP_ROOT, 'static', 'results')
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'webp'} # Allowed image upload types

# Ensure upload and result directories exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)

app = Flask(__name__, static_folder=os.path.join(APP_ROOT, 'static')) # Point static folder correctly
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['RESULT_FOLDER'] = RESULT_FOLDER
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/' # Needed for flashing messages (change in production)

# --- Helper Functions ---
def allowed_file(filename):
    """Checks if the uploaded file extension is allowed."""
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def run_generation(prompt, input_image_path=None):
    """
    Runs the generate.py script with the given arguments.
    Returns (output_image_url_path, error_message, error_details)
    Note: Returns URL *path* relative to static folder, not full URL
    """
    unique_id = str(uuid.uuid4())
    output_filename = f"{unique_id}.png" # Assume PNG output
    # Use absolute path for the subprocess command
    output_image_path_abs = os.path.join(app.config['RESULT_FOLDER'], output_filename)
    # This is the path relative to static folder needed for url_for
    output_image_url_path = os.path.join('results', output_filename)

    # --- Build the command for generate.py ---
    cmd = [
        'python3',
        os.path.join(APP_ROOT, 'generate.py'), # Use absolute path to generate.py
        '--prompt', prompt,
        '--output-image', output_image_path_abs, # Pass absolute path to script
        # Add other default arguments you want here:
        '--compiler', 'nexfort', # Keep nexfort for GPU usage
        '--steps', '25',         # Adjust default steps
        '--seed', str(int(time.time())), # Use time as a somewhat random seed
        # '--height', '512', # Optional: set defaults if needed
        # '--width', '512',
    ]

    # Add input image argument if provided (absolute path)
    if input_image_path:
        cmd.extend(['--input-image', input_image_path])

    print(f"Running command: {' '.join(cmd)}")

    try:
        # Execute the script
        process = subprocess.run(
            cmd,
            check=True,
            capture_output=True,
            text=True,
            timeout=600 # Increase timeout for Colab (e.g., 10 minutes)
        )
        print("Generation script stdout:")
        print(process.stdout)
        print("Generation script stderr:")
        print(process.stderr)

        # Return the URL path relative to static
        return output_image_url_path, None, None

    except subprocess.CalledProcessError as e:
        print(f"Generation script failed with exit code {e.returncode}")
        print("Stderr:")
        print(e.stderr)
        print("Stdout:")
        print(e.stdout)
        error_message = "Image generation failed."
        error_details = e.stderr or e.stdout or "No output captured."
        if os.path.exists(output_image_path_abs):
            try: os.remove(output_image_path_abs)
            except OSError: pass
        return None, error_message, error_details

    except subprocess.TimeoutExpired as e:
        print("Generation script timed out.")
        print("Stderr before timeout:")
        print(e.stderr)
        print("Stdout before timeout:")
        print(e.stdout)
        error_message = f"Image generation timed out after {e.timeout} seconds."
        return None, error_message, "The process took too long to complete."

    except FileNotFoundError:
        print("Error: generate.py or python3 not found.")
        error_message = "Server configuration error: Generation script not found."
        return None, error_message, "Please ensure generate.py is in the correct location."

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        error_message = "An unexpected server error occurred during generation."
        return None, error_message, str(e)


# --- Flask Routes ---
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        prompt = request.form.get('prompt', '').strip()
        image_file = request.files.get('input_image')

        if not prompt:
            flash("Prompt is required.", "error")
            return render_template('index.html', error="Prompt is required.")

        input_image_path_abs = None
        input_image_filename_orig = None

        if image_file and image_file.filename != '':
            if allowed_file(image_file.filename):
                original_filename = secure_filename(image_file.filename)
                unique_suffix = str(uuid.uuid4())[:8]
                input_filename = f"{unique_suffix}_{original_filename}"
                # Save to absolute path
                input_image_path_abs = os.path.join(app.config['UPLOAD_FOLDER'], input_filename)
                try:
                    image_file.save(input_image_path_abs)
                    input_image_filename_orig = original_filename
                    print(f"Input image saved to: {input_image_path_abs}")
                except Exception as e:
                    print(f"Error saving uploaded image: {e}")
                    flash(f"Error saving uploaded image: {e}", "error")
                    return render_template('index.html', error=f"Could not save uploaded image.", prompt=prompt)
            else:
                flash("Invalid image file type. Allowed types: png, jpg, jpeg, webp", "error")
                return render_template('index.html', error="Invalid image file type.", prompt=prompt)

        print("Starting generation process...")
        output_image_url_path, error_message, error_details = run_generation(prompt, input_image_path_abs)
        print("Generation process finished.")

        final_image_url = None
        if output_image_url_path:
            # Generate the full URL using url_for AFTER generation succeeds
            final_image_url = url_for('static', filename=output_image_url_path)

        timestamp = int(time.time())
        return render_template('index.html',
                               image_url=final_image_url,
                               error=error_message,
                               error_details=error_details,
                               prompt=prompt,
                               input_image_filename=input_image_filename_orig,
                               timestamp=timestamp,
                               loading=False
                               )

    # GET Request
    return render_template('index.html', prompt=None, loading=False)

# --- Add static file serving route if needed (Flask usually handles this with url_for) ---
# Though url_for should work with the static_folder config, sometimes explicitly adding helps
# @app.route('/static/<path:filename>')
# def static_files(filename):
#    return send_from_directory(app.static_folder, filename)


# --- Run the App (Specific setup for Colab with ngrok) ---
# This part will be executed in a separate cell later
def run_app():
     # Use host='0.0.0.0' to listen on all interfaces within the container
     # Use a specific port
     app.run(host='0.0.0.0', port=5001) # Don't use debug=True in the final Colab run usually

# We define run_app but don't call it here. We'll call it in the execution cell.

Writing app.py


In [6]:
# Install Flask and ngrok wrapper
!pip install -q Flask pyngrok

# Install ML dependencies (copy from your previous successful install)
!python3 -m pip install -q -U torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0
!python3 -m pip install -q -U diffusers transformers accelerate Pillow matplotlib pandas numpy
!python3 -m pip install -q -U nexfort torchao # Install latest torchao
!python3 -m pip install -q --pre onediff onediffx
# !python3 -m pip install -q oneflow # Optional

In [ ]:
from pyngrok import ngrok, conf
import os

# --- ngrok Configuration ---
# !!! PASTE YOUR NGROK AUTHTOKEN BELOW !!!
NGROK_AUTH_TOKEN = "2waSJAycUk9SskHq1nASfIHCw6A_7gpnRAksSww8dN7BAbRCt" # <--- PASTE YOUR TOKEN HERE
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Set region if needed (e.g., 'us', 'eu', 'ap', 'au', 'sa', 'jp', 'in')
# conf.get_default().region = 'us' # Usually not needed, but uncomment if you have issues

# Import the app object from your app.py file
from app import app

# Terminate any existing ngrok tunnels
ngrok.kill()

# --- Start ngrok tunnel ---
try:
    # Use port 5001 (or whichever port you specified in app.run)
    PORT = 5001
    public_url = ngrok.connect(PORT, "http").public_url
    print("=" * 80)
    print(f"✅ DreamDoodle is running! Access it publicly at: {public_url}")
    print("=" * 80)
    print("NOTE: The first image generation might take several minutes due to model loading/compilation.")
    print("Keep this Colab cell running to keep the app alive.")
    print("Close the tunnel by stopping this cell (Ctrl+C doesn't always work reliably).")

    # --- Run the Flask app ---
    app.run(host='0.0.0.0', port=PORT) # Don't use debug=True here normally

except Exception as e:
    print(f"❌ Failed to start ngrok or Flask app: {e}")
    print("Check your ngrok setup (authtoken?) and Flask app code.")

✅ DreamDoodle is running! Access it publicly at: https://5488-34-83-17-56.ngrok-free.app
NOTE: The first image generation might take several minutes due to model loading/compilation.
Keep this Colab cell running to keep the app alive.
Close the tunnel by stopping this cell (Ctrl+C doesn't always work reliably).
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:43:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:43:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:43:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:45:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:45:52] "GET / HTTP/1.1" 200 -


Input image saved to: /content/DreamDoodleWebApp/static/uploads/553bfe2f_Screenshot_2025-05-04_151641.png
Starting generation process...
Running command: python3 /content/DreamDoodleWebApp/generate.py --prompt add a man standing besides him --output-image /content/DreamDoodleWebApp/static/results/2660099d-b7a5-4ff6-9eb3-455185b730e9.png --compiler nexfort --steps 25 --seed 1746352178 --input-image /content/DreamDoodleWebApp/static/uploads/553bfe2f_Screenshot_2025-05-04_151641.png


INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:53:01] "POST / HTTP/1.1" 200 -


Generation script stdout:
Detected Image-to-Image task (input image provided).
Using AutoPipelineForImage2Image.

Loading model: SG161222/RealVisXL_V4.0
Using scheduler: EulerAncestralDiscreteScheduler
Pipeline moved to device: cuda
Auto-detected resolution: 1024x1024
Adjusted resolution to be multiples of 8: 1024x1024

Applying compiler: nexfort
Using default compiler config: {'mode': 'max-optimize:max-autotune:freezing', 'memory_format': 'channels_last'}
Error during Nexfort setup: The torch version(torch==2.4.0+cu121) of nexfort's compilation environment conflicts with the current environment(torch==2.3.0+cu121)!
You can handle this exception in one of two ways:
1. Reinstall nextort using one of the following commands:
   a. For CN users
      python3 -m pip uninstall nexfort -y && python3 -m pip --no-cache-dir install --pre nexfort -f https://nexfort-whl.oss-cn-beijing.aliyuncs.com/torch2.3.0/cu121/
   b. For NA/EU users
      python3 -m pip uninstall nexfort -y && python3 -m pip -

...

Starting generation process...
Running command: python3 /content/DreamDoodleWebApp/generate.py --prompt image of a cat standing on human lap --output-image /content/DreamDoodleWebApp/static/results/01742ac9-3dd0-4b70-81b3-54f5940ca46c.png --compiler nexfort --steps 25 --seed 1746352422


INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:55:53] "POST / HTTP/1.1" 200 -


Generation script stdout:
Detected Text-to-Image task (no input image provided).
Using AutoPipelineForText2Image.

Loading model: SG161222/RealVisXL_V4.0
Using scheduler: EulerAncestralDiscreteScheduler
Pipeline moved to device: cuda
Auto-detected resolution: 1024x1024
Adjusted resolution to be multiples of 8: 1024x1024

Applying compiler: nexfort
Using default compiler config: {'mode': 'max-optimize:max-autotune:freezing', 'memory_format': 'channels_last'}
Error during Nexfort setup: The torch version(torch==2.4.0+cu121) of nexfort's compilation environment conflicts with the current environment(torch==2.3.0+cu121)!
You can handle this exception in one of two ways:
1. Reinstall nextort using one of the following commands:
   a. For CN users
      python3 -m pip uninstall nexfort -y && python3 -m pip --no-cache-dir install --pre nexfort -f https://nexfort-whl.oss-cn-beijing.aliyuncs.com/torch2.3.0/cu121/
   b. For NA/EU users
      python3 -m pip uninstall nexfort -y && python3 -m pip 

INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:55:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:55:54] "GET /static/results/01742ac9-3dd0-4b70-81b3-54f5940ca46c.png?t=1746352553 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 09:56:40] "GET /static/results/01742ac9-3dd0-4b70-81b3-54f5940ca46c.png HTTP/1.1" 200 -


Starting generation process...
Running command: python3 /content/DreamDoodleWebApp/generate.py --prompt image of a cat standing on human lap --output-image /content/DreamDoodleWebApp/static/results/00bd05ff-825c-438d-8983-718bf1e270b6.png --compiler nexfort --steps 25 --seed 1746352605
